In [ ]:
import redis
import json

# Define connection variables
host = 'redis-13857.c14.us-east-1-2.ec2.redns.redis-cloud.com'
port = 13857
password = 'okNaqK7r5dKE6stSis0FlUuiDlhH2t7M'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')


# Customer Engagement via Social Media Insights

## Description of the use case
This use case involves analyzing social media data to gain insights into customer preferences and engagement. By leveraging these insights, businesses can tailor their marketing strategies and improve customer interactions.

## Redis Data Structures Used
- **Hashes**: To store user profiles and social media interaction data.
- **Streams**: To capture and process real-time social media data.
- **Sorted Sets**: To rank and prioritize social media posts and interactions.

In [ ]:
# Mock data for the use case
mock_data = {
    'user:1001': {'name': 'Alice', 'likes': 10, 'shares': 5, 'comments': 2},
    'user:1002': {'name': 'Bob', 'likes': 15, 'shares': 3, 'comments': 4},
    'user:1003': {'name': 'Charlie', 'likes': 5, 'shares': 2, 'comments': 1},
    'post:2001': {'content': 'Great product!', 'likes': 10, 'shares': 5},
    'post:2002': {'content': 'Highly recommend this!', 'likes': 15, 'shares': 3},
    'post:2003': {'content': 'Good value for money.', 'likes': 5, 'shares': 2}
}

# Use Hashes to store user profiles and social media interaction data
for key, value in mock_data.items():
    r.hset(key, mapping=value)

# Use Streams to capture real-time social media data
r.xadd('social_media_stream', {'user_id': 'user:1001', 'post_id': 'post:2001', 'action': 'like'})
r.xadd('social_media_stream', {'user_id': 'user:1002', 'post_id': 'post:2002', 'action': 'share'})

# Process stream data to update engagement metrics
stream_entries = r.xrange('social_media_stream')
for entry in stream_entries:
    user_id = entry[1]['user_id']
    post_id = entry[1]['post_id']
    action = entry[1]['action']
    if action == 'like':
        r.hincrby(user_id, 'likes', 1)
        r.hincrby(post_id, 'likes', 1)
    elif action == 'share':
        r.hincrby(user_id, 'shares', 1)
        r.hincrby(post_id, 'shares', 1)

# Use Sorted Sets to rank and prioritize social media posts
for post_id in ['post:2001', 'post:2002', 'post:2003']:
    likes = int(r.hget(post_id, 'likes'))
    shares = int(r.hget(post_id, 'shares'))
    score = likes + (shares * 2)  # Example scoring algorithm
    r.zadd('post_ranking', {post_id: score})

# Print the updated user profiles and post engagements
for user_id in ['user:1001', 'user:1002', 'user:1003']:
    print(f"{user_id}: {r.hgetall(user_id)}")

for post_id in ['post:2001', 'post:2002', 'post:2003']:
    print(f"{post_id}: {r.hgetall(post_id)}")

# Print the ranked social media posts
ranked_posts = r.zrange('post_ranking', 0, -1, withscores=True, desc=True)
print('Ranked Social Media Posts:')
for post in ranked_posts:
    print(post)